In [1]:
# !pip install openai
# !pip install playwright
# pip install --upgrade openai

In [2]:
import openai
import os
import subprocess
from PIL import Image
from openai import OpenAIError
from dotenv import load_dotenv

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

TRANSCRIPT_FILE = "transcript-bio.txt"
PROMPT_FILE = "prompt-bio.txt"

# Output Files
CODE_JS_FILE = "sketch.js"
IMAGE_FILE = "img.png"
HTML_FILE = "index.html"
P5_JS_LIBRARY = "p5.js"

In [4]:
client = openai.Client(
    api_key=OPENAI_API_KEY
)

try:
    with open(PROMPT_FILE, "r", encoding="utf-8") as f:
        gpt_prompt = f.read()
    with open(TRANSCRIPT_FILE, "r", encoding="utf-8") as f:
        transcript_content = f.read()
except FileNotFoundError as e:
    print(f"Error: File not found: {e.filename}")
    exit(1)
except Exception as e:
    print(f"Error reading file: {e}")
    exit(1)

try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": gpt_prompt},
            {"role": "user", "content": f"<transcript>{transcript_content}<transcript>"}
        ],
        temperature=0.2,  # Adjust for creativity
    )
    code_js = response.choices[0].message.content
except OpenAIError as e:
    print(f"OpenAI API Error: {e}")
    exit(1)
except Exception as e:
    print(f"Error during GPT interaction: {e}")
    exit(1)

In [5]:
code_js

'```javascript\nfunction setup() {\n  createCanvas(800, 600);\n  background(255);\n  noLoop();\n  setTimeout(() => { saveCanvas(\'img\', \'png\'); document.body.setAttribute("data-ready", "true"); }, 500);\n}\n\nfunction draw() {\n  let colors = [\'#FF6347\', \'#4682B4\', \'#32CD32\', \'#FFD700\'];\n  let bases = [\'A\', \'T\', \'C\', \'G\'];\n  let x = 100;\n  let y = 300;\n  let baseWidth = 50;\n  let baseHeight = 100;\n  \n  for (let i = 0; i < 20; i++) {\n    let index = floor(random(4));\n    fill(colors[index]);\n    rect(x, y - baseHeight / 2, baseWidth, baseHeight);\n    fill(0);\n    textSize(32);\n    textAlign(CENTER, CENTER);\n    text(bases[index], x + baseWidth / 2, y);\n    x += baseWidth + 10;\n  }\n}\n```'

In [6]:
# --- Save p5.js Code ---
import re
cleaned_code = re.sub(r"```javascript\n|```", "", code_js)

try:
    with open(CODE_JS_FILE, "w", encoding="utf-8") as f:
        f.write(cleaned_code)
    print(f"Generated p5.js code saved to: {CODE_JS_FILE}")
except Exception as e:
    print(f"Error saving p5.js code: {e}")
    exit(1)


Generated p5.js code saved to: sketch.js


In [7]:
# import asyncio
# from playwright.async_api import async_playwright

# async def capture_screenshot():
#     async with async_playwright() as p:
#         browser = await p.chromium.launch()
#         page = await browser.new_page()

#         # Open the p5.js sketch URL (Replace with your actual URL)
#         await page.goto("http://localhost:8000")

#         # Wait for the canvas to be ready
#         await page.wait_for_selector("canvas")

#         # Take a screenshot and save it as "screenshot.png"
#         await page.screenshot(path="img.png")

#         await browser.close()
#         print("Diagram saved as 'img.png'")

# # Run the function inside Jupyter Notebook
# await capture_screenshot()

In [8]:
import asyncio
from playwright.async_api import async_playwright

async def capture_image():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()

        # Open the local p5.js sketch
        await page.goto("http://localhost:8000/index.html")

        # Wait for the body to get the 'data-ready' attribute
        await page.wait_for_selector("body[data-ready='true']")

        # Capture the image
        await page.screenshot(path="img_bio.png")

        await browser.close()
        print("Image saved as img.png")

# Run the async function in Jupyter Notebook
await capture_image()

Image saved as img.png
